# Deployment
In this notebook, our objective is to register the model using MLFlow

# Notebook Overview
- Install requirements and Imports Dependencies
- Configurations
- Get Text Data
- Loading Model
- MLFlow - Register Model

# Imports Dependencies 

In [1]:
# Standard Library Imports
import logging
import warnings

# Third-Party Libraries
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# MLflow for Experiment Tracking and Model Management
import mlflow
from mlflow.types.schema import Schema, ColSpec
from mlflow.models import ModelSignature

# Configurations

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# Define global experiment and run names to be used throughout the notebook
REGISTER_NAME = "Shakespeare_Model"
EXPERIMENT_NAME = "Shakespeare Text Generation"
RUN_NAME = "Shakespeare_main"


# Set up the paths
DATA_PATH = "../shakespeare.txt"
MODEL_PATH = 'models/dict_torch_rnn_model.pt'
MODEL_DECODER_PATH = "models/decoder.pt"
MODEL_ENCODER_PATH = "models/encoder.pt"

# Set up the chunk separator for text processing
CHUNK_SEPARATOR = "\n\n"

In [4]:
# === Create logger ===
logger = logging.getLogger("deployment-notebook")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                             datefmt="%Y-%m-%d %H:%M:%S") 

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [5]:
logger.info('Notebook execution started.')

2025-04-11 11:39:39 - INFO - Notebook execution started.


# Get Text Data
This is the text we'll use as a basis for our generations: let's try to generate 'Shakespearean' texts.

This text is from Shakespeare's Sonnet 1. It's one of the 154 sonnets written by William Shakespeare that were first published in 1609. This particular sonnet, like many others, discusses themes of beauty, procreation, and the transient nature of life, urging the beautiful to reproduce so their beauty can live on through their offspring.

In [6]:
with open(DATA_PATH,'r',encoding='utf8') as f:
    text = f.read()
all_characters = set(text)

# Loading Model
The models are available at the [models](models/) folder, where:
 - [Tensorflow Jupyter Notebook](RNN_for_text_generation_TF.ipynb): `tf_rnn_model.h5`
 - [PyTorch Jupyter Notebook](RNN_for_text_generation_Torch.): `dict_torch_rnn_model.pt`. Also includes the `decoder.pt` and `encoder.pt`

In [7]:
class CharModel(nn.Module):
    def __init__(self, decoder, encoder, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5, use_gpu=False):
        """Initializes CharModel

        Args:
            decoder: Assigns a unique integer to each character in a dictionary format
            encoder : Reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers.
            all_chars: Set of unique characters found in the text.
            num_hidden: Number of hidden layers. Defaults to 256.
            num_layers: Number of layers. Defaults to 4.
            drop_prob: Regularization technique to prevent overfitting. Defaults to 0.5.
            use_gpu: If the model uses GPU. Defaults to False.
        """
        try:
            super().__init__()
            self.drop_prob = drop_prob
            self.num_layers = num_layers
            self.num_hidden = num_hidden
            self.use_gpu = use_gpu
            
            self.all_chars = all_chars
            self.decoder = torch.load(decoder)
            self.encoder = torch.load(encoder)
            
            self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
            self.dropout = nn.Dropout(drop_prob)
            self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))

            logger.info("CharModel Initialized successfully")
    
        except Exception as e:
            logger.error(f"Error Initializing CharModel: {str(e)}")
      
    
    def forward(self, x, hidden):
        """Implementation of the CharModel logic, in which, the input passes through every step of the arquiteture

        Args:
            x: Input tensor with shape (batch size and senquency length) containing character indices.
            hidden: Tuple containing the inicial hidden states of the CharModel each with shape (batch size and senquency length).

        Returns:
            final_out: Output tensor representing the predicted logits for each character in the sequence.
            hidden: Tuple containing the final hidden states of the CharModel.
        """
        try:
            lstm_output, hidden = self.lstm(x, hidden)       
            drop_output = self.dropout(lstm_output)
            drop_output = drop_output.contiguous().view(-1, self.num_hidden)
            final_out = self.fc_linear(drop_output)
            
            return final_out, hidden
        
        except Exception as e:
            logger.error(f"Error implementing CharModel logic: {str(e)}")
    
    
    def hidden_state(self, batch_size):
        """
        Initializes and returns the initial hidden state for a recurrent neural network .

        This method creates zero-filled tensors for the hidden state (h_0) and cell state (c_0), 
        supporting GPU execution if `self.use_gpu` is set to True.

        Args:
            batch_size: The number of sequences in the input batch, used to determine the tensor dimensions.

        Returns:
            Tuple: A tuple containing the hidden state and cell state tensors 
            with shape (num_layers, batch_size, num_hidden). Returns None if an exception occurs, and logs the error.
        """
        try:
            if self.use_gpu:
                hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                        torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
            else:
                hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                        torch.zeros(self.num_layers,batch_size,self.num_hidden))
            
            return hidden
        except Exception as e:
            logger.error(f"Error hidding state: {str(e)}")


# MLFlow - Register Model

In [8]:
class RNNModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Loads the model and associated artifacts (encoder, decoder) into memory.

        Args:
            context: MLflow context containing paths to model artifacts.
        """
        try:
            self.model = CharModel(
                            all_chars=all_characters,
                            num_hidden=512,
                            num_layers=3,
                            drop_prob=0.5,
                            use_gpu=False,
                            decoder=context.artifacts['decoder'],
                            encoder=context.artifacts['encoder']
                                            
                        )

            self.model.load_state_dict(torch.load(context.artifacts['model_state_dict']))
            self.model.eval()
            logger.info("Loading context done successfully")
        except Exception as e:
            logger.error(f"Error loading context: {str(e)}")

    def one_hot_encoder(self, encoded_text, num_uni_chars):
        """
        Convert categorical data into a fixed-size vector of numerical values.

        Args:
            encoded_text: Batch of encoded text.
            num_uni_chars: Number of unique characters

        """
        try:
            one_hot = np.zeros((encoded_text.size, num_uni_chars))
            one_hot = one_hot.astype(np.float32)
            one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
            one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
            
            return one_hot
        
        except Exception as e:
            logger.error(f"Error converting categorical data: {str(e)}")

    def predict_next_char(self, char, hidden=None, k=3):
        """
        Predicts the next character given an input character and the current hidden state.

        This method encodes the input character, feeds it through the trained character-level 
        language model (e.g., LSTM), and samples from the top-k most probable characters 
        to determine the next one. It also returns the updated hidden state for sequential prediction.

        Args:
            char: The input character to start prediction from.
            hidden: Current hidden state of the model. Each tensor has shape (num_layers, batch_size, num_hidden).
                If None, a new hidden state should be initialized before calling this method.
            k: Number of top predictions to sample from.

        Returns:
            A tuple containing the predicted next character and the updated hidden state.
        """
        try:
            encoded_text = self.model.encoder[char]
            encoded_text = np.array([[encoded_text]])
            encoded_text = self.one_hot_encoder(encoded_text, len(self.model.all_chars))
            inputs = torch.from_numpy(encoded_text)
            inputs = inputs.cpu()
                
            hidden = tuple([state.data for state in hidden])
            lstm_out, hidden = self.model(inputs, hidden)    
            probs = F.softmax(lstm_out, dim=1).data
            probs = probs.cpu()

            
            probs, index_positions = probs.topk(k)        
            index_positions = index_positions.numpy().squeeze()
            probs = probs.numpy().flatten()
            probs = probs/probs.sum()
            char = np.random.choice(index_positions, p=probs)

            return self.model.decoder[char], hidden
        except Exception as e:
            logger.error(f"Error predicting next char: {str(e)}")

    def generate_text(self, seed, size, k=3):
        """
        Generates a sequence of text using the trained character-level language model.

        Starting from a seed string, this method uses the model to predict the next character
        one at a time, feeding each predicted character back into the model. It continues
        this process until the desired output length is reached.

        Args:
            seed: The initial sequence of characters used to start the text generation.
            size: The number of characters to generate after the seed.
            k: Number of top character predictions to consider for sampling at each step.

        Returns:
            The full generated text including the seed and the newly predicted characters.
        """
        try:
            self.model.cpu()
                
            self.model.eval()
            output_chars = [c for c in seed]
            hidden = self.model.hidden_state(1)
            
            for char in seed:
                char, hidden = self.predict_next_char(char, hidden, k=k)
        
            output_chars.append(char)
            for i in range(size):
                char, hidden = self.predict_next_char(output_chars[-1], hidden, k=k)
                output_chars.append(char)
                
            return ''.join(output_chars)
        
        except Exception as e:
            logger.error(f"Error generating text: {str(e)}")
        
    def predict(self, context, model_input):
        """
        Runs inference using the loaded model and input data.

        Args:
            context: The MLflow context object.
            model_input : A dictionary containing 'seed' and 'size' keys.

        Returns:
             The output from the model containing the predicted text.
        """
        try:
            initial_word = model_input['initial_word'][0]
            size = model_input['size'][0]
            output = self.generate_text(seed=initial_word, size=size)
            
            return output
        except Exception as e:
            logger.error(f"Error predicting text: {str(e)}")

    @classmethod
    def log_model(cls, model_state_dict, decoder, encoder, demo_folder="../demo"): 
        """
        Logs the model to MLflow, including artifacts, dependencies, and input/output signatures.

        Args:
            model_state_dict: Path where the model is saved before logging.
            decoder: Assigns a unique integer to each character in a dictionary format
            encoder : Reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers.
            demo_folder: Path to the folder containing the compiled demo UI. Defaults to "demo".".
        """
        try:
            input_schema = Schema(
                [
                    ColSpec("string", "initial_word"),
                    ColSpec("long", "size")
                ]
            )

            output_schema = Schema(
                [
                    ColSpec("string", "generated_text")
                ]
            )
        
            signature = ModelSignature(inputs=input_schema, outputs=output_schema)
                
            requirements = [
                "torch",
                "numpy"
            ]
            mlflow.pyfunc.log_model(
                model_state_dict,
                python_model=cls(),
                artifacts={
                    "model_state_dict": model_state_dict, 
                    'decoder': decoder, 
                    'encoder': encoder, 
                    "demo": demo_folder},
                signature=signature,
                pip_requirements=requirements
            )
            logger.info("Logging model to MLflow done successfully")

        except Exception as e:
            logger.error(f"Error logging model to MLflow: {str(e)}")

In [9]:
mlflow.set_experiment(experiment_name= EXPERIMENT_NAME)

<Experiment: artifact_location='/phoenix/mlflow/238715752359479816', creation_time=1743590713735, experiment_id='238715752359479816', last_update_time=1743590713735, lifecycle_stage='active', name='Shakespeare Text Generation', tags={}>

In [10]:
model_state_dict = MODEL_PATH

In [11]:
register_name = REGISTER_NAME 

In [12]:
with mlflow.start_run(run_name = RUN_NAME) as run:
    logger.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    RNNModel.log_model(model_state_dict, MODEL_DECODER_PATH, MODEL_ENCODER_PATH)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/{model_state_dict}", name=register_name)

2025-04-11 11:39:39 - INFO - Run's Artifact URI: /phoenix/mlflow/238715752359479816/e3adf895dc1e4e50977177e26a631b20/artifacts


2025-04-11 11:39:40 - INFO - Logging model to MLflow done successfully
Registered model 'Shakespeare_Model' already exists. Creating a new version of this model...
Created version '12' of model 'Shakespeare_Model'.


In [13]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions(register_name, stages=["None"])
latest_model_version = model_metadata[0].version
latest_model_version

12

## Testing registered model

In [14]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/{register_name}/{latest_model_version}")
print(model.predict({"initial_word": 'Love ', "size": 100}))

2025-04-11 11:39:41 - INFO - CharModel Initialized successfully
2025-04-11 11:39:41 - INFO - Loading context done successfully


Love of the word
    Would that the caln of the sare ant he sare,
    The cound the will a mester, will to


Built with ❤️ using Z by HP AI Studio.